### Import library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Read csv file

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
df.shape

In [ ]:
df.iloc[0:5, 0:5]

### Wide to long Dataframe

In [ ]:
df_melt=pd.melt(df, id_vars=['Province/State', 'Country/Region', 'Lat', 
                             'Long'], var_name='date', 
                value_name='confirmed_case')
df_melt.head()

In [ ]:
df_melt.info()

In [ ]:
df_melt['date']=pd.to_datetime(df_melt.date)

In [ ]:
df_melt.head()

In [ ]:
df_melt.columns=df_melt.columns.str.replace('/','_')

In [ ]:
df_melt.columns

### Bangladesh

In [ ]:
df_bd=df_melt[df_melt.Country_Region=='Bangladesh']
df_bd.shape

In [ ]:
df_bd.head()

In [ ]:
df_bd['daily_case']=df_bd.confirmed_case.diff().fillna(0)

In [ ]:
df_bd.tail()

In [ ]:
df_bd.daily_case.max()

In [ ]:
fig, ax=plt.subplots(figsize=(25,10))
ax.grid(linewidth=0.4, color='#8f8f8f') 


ax.plot(df_bd.date,df_bd.daily_case ,color='r', linewidth=1.5)

ax.set_facecolor("#FFFBF3") 
ax.set_xlabel('\nDate',size=25,color='#4bb4f2')
ax.set_ylabel('Number of Confirmed Cases\n',
              size=25,color='#4bb4f2')

In [ ]:
bd_monthly=df_bd.groupby([df_bd.date.dt.year,
               df_bd.date.dt.month])['daily_case'].sum()
bd_monthly

In [ ]:
bd_2020=bd_monthly[2020]
bd_2020.index=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
bd_2020

In [ ]:
fig, ax=plt.subplots()
#'#003262'
ax.bar(bd_2020.index,bd_2020, color='#8B072F')

In [ ]:
bd_2021=bd_monthly[2021]
bd_2021.index=['Jan', 'Feb', 'Mar', 'Apr', "May", "Jun", "Jul", "Aug"]
bd_2021

In [ ]:
fig, ax=plt.subplots(figsize=(6,6))
ax.bar(bd_2021.index,bd_2021, color='#00274C')

### `resample` method

In [ ]:
df_bd.set_index('date', inplace=True)
df_bd.head()

In [ ]:
df_bd.daily_case.resample('M').sum()

In [ ]:
df_bd.daily_case.resample('M').sum().plot(kind='bar')

### Stationarity check

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(df_bd.daily_case)

In [ ]:
result = adfuller(df_bd.daily_case)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

if result[0] < result[4]["5%"]:
    print ("Reject Ho - Time Series is Stationary")
else:
    print ("Failed to Reject Ho - Time Series is Non-Stationary")

In [2]:
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, HoverTool, DatetimeTickFormatter
from bokeh.layouts import column
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

In [3]:
output_notebook()

Loading BokehJS ...

In [6]:
def modify_doc(doc):
    raw=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    raw_melt=pd.melt(raw, id_vars=['Province/State', 'Country/Region', 'Lat', 
                                 'Long'], var_name='date', 
                    value_name='confirmed_case')
    raw_melt["date"]=pd.to_datetime(raw_melt.date)
    df=pd.pivot_table(raw_melt, index="date", columns="Country/Region", 
                      values="confirmed_case",
                      aggfunc='sum')
    df=df.diff().dropna()
    bangladesh=pd.DataFrame(df.Bangladesh).reset_index()
    source=ColumnDataSource(dict(date=bangladesh.date, daily_case=bangladesh.iloc[:,-1]))
    hover=HoverTool(tooltips=[("date", "@date{%F}"), 
                        ("daily case", "@daily_case")], 
    formatters={"@date" : "datetime"}, mode="mouse")
    country_list=df.columns.tolist()

    p=figure(plot_width=600, plot_height=300, x_axis_type="datetime", 
             sizing_mode="stretch_both", 
             tools=[hover, "pan", "reset", "save"])
    p.line(x="date", y="daily_case", source=source, color="DarkBlue", 
          line_width=1.5)
    select=Select(title="Country", options= country_list, 
                  value="Bangladesh", background="orange")
    
    p.xaxis.axis_label="Time"
    p.xaxis.axis_label_text_color="DarkGreen"
    p.xaxis.formatter=DatetimeTickFormatter(months="%b %y")

    p.yaxis.axis_label="Daily infection"
    p.yaxis.axis_label_text_color="DarkGreen"

    p.title.text="Daily Covid case in Bangladesh"
    p.title.text_font="ecotype"
    p.title.text_color="DarkGreen"
    p.title.text_font_size="20px"

    def update_data(attr, old, new):
        desired_country=select.value
        p.title.text=f"Daily Covid case in {desired_country}"
        source_data=pd.DataFrame(df[desired_country]).reset_index()
        source.data=dict(date=source_data.date, daily_case=source_data.iloc[:,-1])

    select.on_change("value", update_data)
    inputs=column(select, p)
    doc.add_root(inputs)
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app, notebook_url="http://localhost:8888")